In [1]:
# 필요한 라이브러리 import
import os
import pandas as pd
from pathlib import Path
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq
import glob
import traceback
from multiprocessing import Pool, cpu_count
from functools import partial
from collections import defaultdict
from tqdm import tqdm

## 1. 파일 비교 분석

두 디렉토리에 있는 동일한 이름의 SAS 파일들을 비교하여 칼럼과 값의 차이점을 분석합니다.

### 비교 대상
- **디렉토리 1**: `/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date`
- **디렉토리 2**: `/home/hashjamm/project_data/disease_network/sas_files/matched`

### 비교 항목
1. **칼럼 차이**: 각 파일의 칼럼 구조 비교
   - dir1에만 있는 칼럼
   - dir2에만 있는 칼럼
   - 공통 칼럼

2. **값 차이**: 공통 칼럼의 데이터 값 비교
   - 비교 조건: 행 수가 동일하고 `PERSON_ID` 종류가 동일한 경우에만 수행
   - `PERSON_ID` 기준으로 정렬 후 값 비교
   - 차이가 있는 칼럼 식별

### 출력 결과
- 칼럼 차이가 있는 파일 수 및 차이 패턴
- 값 차이가 있는 파일 수 (행 수 동일 + person_id 종류 동일)
- 행 수가 다른 파일 수
- person_id 종류가 다른 파일 수 (행 수 동일)
- 칼럼과 값이 모두 동일한 파일 수


In [21]:
# 디렉토리 경로 설정
dir1 = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
dir2 = '/home/hashjamm/project_data/disease_network/sas_files/matched'

# 각 디렉토리의 파일 목록 가져오기
files1 = set([f for f in os.listdir(dir1) if f.endswith('.sas7bdat')])
files2 = set([f for f in os.listdir(dir2) if f.endswith('.sas7bdat')])

# 공통 파일명 찾기
common_files = sorted(files1 & files2)

print(f"디렉토리 1의 파일 수: {len(files1)}")
print(f"디렉토리 2의 파일 수: {len(files2)}")
print(f"공통 파일 수: {len(common_files)}")
print(f"\n공통 파일 목록 (처음 10개): {common_files[:10]}")


디렉토리 1의 파일 수: 1187
디렉토리 2의 파일 수: 1187
공통 파일 수: 1187

공통 파일 목록 (처음 10개): ['matched_a01.sas7bdat', 'matched_a02.sas7bdat', 'matched_a03.sas7bdat', 'matched_a04.sas7bdat', 'matched_a05.sas7bdat', 'matched_a06.sas7bdat', 'matched_a07.sas7bdat', 'matched_a08.sas7bdat', 'matched_a09.sas7bdat', 'matched_a15.sas7bdat']


In [22]:
# 칼럼 차이와 값 차이를 명확히 확인하는 코드
dir1 = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
dir2 = '/home/hashjamm/project_data/disease_network/sas_files/matched'

# 공통 파일 목록 가져오기
files1 = set([f for f in os.listdir(dir1) if f.endswith('.sas7bdat')])
files2 = set([f for f in os.listdir(dir2) if f.endswith('.sas7bdat')])
common_files = sorted(files1 & files2)

# 각 파일에 대한 상세 비교 결과 저장
detailed_results = []

for filename in common_files:
    file1_path = os.path.join(dir1, filename)
    file2_path = os.path.join(dir2, filename)
    
    try:
        df1 = pd.read_sas(file1_path, encoding='utf-8')
        df2 = pd.read_sas(file2_path, encoding='utf-8')
        
        # 칼럼 차이 확인
        cols1 = set(df1.columns)
        cols2 = set(df2.columns)
        only_in_dir1 = cols1 - cols2
        only_in_dir2 = cols2 - cols1
        common_cols = cols1 & cols2
        
        result = {
            'filename': filename,
            'dir1_cols': sorted(list(cols1)),
            'dir2_cols': sorted(list(cols2)),
            'only_in_dir1': sorted(list(only_in_dir1)),
            'only_in_dir2': sorted(list(only_in_dir2)),
            'common_cols': sorted(list(common_cols)),
            'has_col_diff': len(only_in_dir1) > 0 or len(only_in_dir2) > 0,
            'dir1_rows': len(df1),
            'dir2_rows': len(df2)
        }
        
        # person_id 칼럼 확인
        person_id_col = None
        for col_name in ['PERSON_ID', 'person_id', 'Person_ID']:
            if col_name in common_cols:
                person_id_col = col_name
                break
        
        # 공통 칼럼이 있고 행 수가 같고 person_id 종류가 같은 경우 값 비교
        if len(common_cols) > 0 and len(df1) == len(df2) and person_id_col:
            # person_id 종류 확인
            person_id1_set = set(df1[person_id_col].dropna().unique())
            person_id2_set = set(df2[person_id_col].dropna().unique())
            person_id_same = person_id1_set == person_id2_set
            
            if person_id_same:
                # 공통 칼럼만 선택
                df1_common = df1[list(common_cols)].copy()
                df2_common = df2[list(common_cols)].copy()
                
                # person_id 기준으로 정렬 후 비교
                df1_sorted = df1_common.sort_values(by=[person_id_col] + [c for c in common_cols if c != person_id_col]).reset_index(drop=True)
                df2_sorted = df2_common.sort_values(by=[person_id_col] + [c for c in common_cols if c != person_id_col]).reset_index(drop=True)
                
                # 값 비교
                values_equal = df1_sorted.equals(df2_sorted)
                
                if not values_equal:
                    # 어느 칼럼에서 차이가 나는지 확인
                    diff_cols = []
                    for col in common_cols:
                        if not df1_sorted[col].equals(df2_sorted[col]):
                            diff_cols.append(col)
                    result['value_diff_cols'] = diff_cols
                else:
                    result['value_diff_cols'] = []
                
                result['values_equal'] = values_equal
                result['rows_equal'] = True
                result['person_id_same'] = True
            else:
                result['values_equal'] = None
                result['rows_equal'] = True
                result['person_id_same'] = False
                result['value_diff_cols'] = None
        elif len(common_cols) > 0:
            # 행 수가 다른 경우
            result['values_equal'] = None
            result['rows_equal'] = False
            result['person_id_same'] = None
            result['value_diff_cols'] = None
        else:
            # 공통 칼럼이 없는 경우
            result['values_equal'] = None
            result['rows_equal'] = None
            result['person_id_same'] = None
            result['value_diff_cols'] = None
        
        detailed_results.append(result)
        
    except Exception as e:
        detailed_results.append({
            'filename': filename,
            'error': str(e)
        })

print(f"비교 완료: {len(detailed_results)}개 파일 처리됨\n")


비교 완료: 1187개 파일 처리됨



In [23]:
# 결과를 요약 테이블로 정리
summary_list = []
for result in detailed_results:
    if 'error' not in result:
        summary_list.append({
            'filename': result['filename'],
            'dir1_칼럼수': len(result['dir1_cols']),
            'dir2_칼럼수': len(result['dir2_cols']),
            '공통_칼럼수': len(result['common_cols']),
            'dir1에만있는칼럼': ', '.join(result['only_in_dir1']) if result['only_in_dir1'] else '',
            'dir2에만있는칼럼': ', '.join(result['only_in_dir2']) if result['only_in_dir2'] else '',
            '칼럼차이': '예' if result['has_col_diff'] else '아니오',
            'dir1_행수': result['dir1_rows'],
            'dir2_행수': result['dir2_rows'],
            '행수동일': '예' if result['dir1_rows'] == result['dir2_rows'] else '아니오',
            'person_id동일': '예' if result.get('person_id_same') == True else ('아니오' if result.get('person_id_same') == False else '비교불가'),
            '값동일': '예' if result.get('values_equal') == True else ('아니오' if result.get('values_equal') == False else '비교불가'),
            '값차이칼럼': ', '.join(result.get('value_diff_cols', [])) if result.get('value_diff_cols') else ''
        })
    else:
        summary_list.append({
            'filename': result['filename'],
            '오류': result['error']
        })

summary_df = pd.DataFrame(summary_list)


In [24]:
# 간단한 요약 결과
print("=" * 80)
print("📊 파일 비교 결과 요약")
print("=" * 80)

total_files = len(detailed_results)
col_diff_count = len(summary_df[summary_df['칼럼차이'] == '예'])
value_diff_count = len(summary_df[(summary_df['행수동일'] == '예') & (summary_df['person_id동일'] == '예') & (summary_df['값동일'] == '아니오')])
identical_count = len(summary_df[(summary_df['칼럼차이'] == '아니오') & (summary_df['행수동일'] == '예') & (summary_df['person_id동일'] == '예') & (summary_df['값동일'] == '예')])
row_diff_count = len(summary_df[summary_df['행수동일'] == '아니오'])
person_id_diff_count = len(summary_df[(summary_df['행수동일'] == '예') & (summary_df['person_id동일'] == '아니오')])

print(f"\n총 비교 파일 수: {total_files}개")
print(f"\n📋 칼럼 차이:")
print(f"   • 칼럼이 다른 파일: {col_diff_count}개 ({col_diff_count/total_files*100:.1f}%)")

if col_diff_count > 0:
    # 칼럼 차이 패턴 분석
    dir1_only_cols = set()
    dir2_only_cols = set()
    for result in detailed_results:
        if 'error' not in result and result['has_col_diff']:
            dir1_only_cols.update(result['only_in_dir1'])
            dir2_only_cols.update(result['only_in_dir2'])
    
    print(f"   • dir1에만 있는 칼럼: {sorted(dir1_only_cols)}")
    print(f"   • dir2에만 있는 칼럼: {sorted(dir2_only_cols)}")

print(f"\n📊 값 차이 (행 수 동일 + person_id 종류 동일):")
print(f"   • 값이 다른 파일: {value_diff_count}개 ({value_diff_count/total_files*100:.1f}%)")
print(f"   • 행 수가 다른 파일: {row_diff_count}개 ({row_diff_count/total_files*100:.1f}%)")
print(f"   • person_id 종류가 다른 파일 (행 수 동일): {person_id_diff_count}개 ({person_id_diff_count/total_files*100:.1f}%)")

print(f"\n✅ 동일한 파일:")
print(f"   • 칼럼과 값 모두 동일: {identical_count}개 ({identical_count/total_files*100:.1f}%)")


📊 파일 비교 결과 요약

총 비교 파일 수: 1187개

📋 칼럼 차이:
   • 칼럼이 다른 파일: 1187개 (100.0%)
   • dir1에만 있는 칼럼: ['DTH_CODE1', 'DTH_CODE2', 'DTH_YM', 'index_date', 'index_key_seq', 'matched_id']
   • dir2에만 있는 칼럼: ['AGE_GROUP', 'CTRB_PT_TYPE_CD', 'SEX', 'SGG']

📊 값 차이 (행 수 동일 + person_id 종류 동일):
   • 값이 다른 파일: 0개 (0.0%)
   • 행 수가 다른 파일: 0개 (0.0%)
   • person_id 종류가 다른 파일 (행 수 동일): 0개 (0.0%)

✅ 동일한 파일:
   • 칼럼과 값 모두 동일: 0개 (0.0%)


In [26]:
# outcome_fu_1.sas7bdat ~ outcome_fu_10.sas7bdat 파일들의 상위 5개 행 확인
base_dir = '/home/hashjamm/project_data/disease_network/sas_files'

for i in range(1, 11):
    file_path = os.path.join(base_dir, f'outcome_fu_{i}.sas7bdat')
    
    print("=" * 80)
    print(f"파일: outcome_fu_{i}.sas7bdat")
    print("=" * 80)
    
    try:
        df = pd.read_sas(file_path, encoding='utf-8')
        print(f"총 행 수: {len(df)}, 총 열 수: {len(df.columns)}")
        print(f"\n칼럼 목록: {list(df.columns)}")
        print("\n상위 5개 행:")
        print(df.head(5))
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
    except Exception as e:
        print(f"오류 발생: {e}")
    
    print("\n")


파일: outcome_fu_1.sas7bdat
총 행 수: 3311623, 총 열 수: 2

칼럼 목록: ['PERSON_ID', 'abb_sick']

상위 5개 행:
    PERSON_ID abb_sick
0  10000025.0      J00
1  10000025.0      J01
2  10000025.0      J02
3  10000025.0      J20
4  10000025.0      L08


파일: outcome_fu_2.sas7bdat
총 행 수: 5896267, 총 열 수: 2

칼럼 목록: ['PERSON_ID', 'abb_sick']

상위 5개 행:
    PERSON_ID abb_sick
0  10000025.0      B08
1  10000025.0      D69
2  10000025.0      J00
3  10000025.0      J01
4  10000025.0      J02


파일: outcome_fu_3.sas7bdat
총 행 수: 8229879, 총 열 수: 2

칼럼 목록: ['PERSON_ID', 'abb_sick']

상위 5개 행:
    PERSON_ID abb_sick
0  10000025.0      B08
1  10000025.0      D69
2  10000025.0      J00
3  10000025.0      J01
4  10000025.0      J02


파일: outcome_fu_4.sas7bdat
총 행 수: 10550189, 총 열 수: 2

칼럼 목록: ['PERSON_ID', 'abb_sick']

상위 5개 행:
    PERSON_ID abb_sick
0  10000025.0      B08
1  10000025.0      D69
2  10000025.0      H52
3  10000025.0      H60
4  10000025.0      H68


파일: outcome_fu_5.sas7bdat
총 행 수: 12766765, 총 열 수: 2

칼럼 목록:

## 2. 누적 파일 검증: fu_{n+1} == dt_year_{n+1} + fu_n

`outcome_fu_{n+1}` 파일이 `outcome_dt_year_{n+1}`과 `outcome_fu_n`을 합친 것과 동일한지 확인합니다.

### 비교 대상
- **fu_2** == dt_year_2 + fu_1
- **fu_3** == dt_year_3 + fu_2
- **fu_4** == dt_year_4 + fu_3
- ...
- **fu_10** == dt_year_10 + fu_9

### 비교 항목
1. **칼럼 비교**: 각 파일의 칼럼 구조가 동일한지 확인
2. **행 수 비교**: 합친 결과와 fu_{n+1}의 행 수가 동일한지 확인
3. **값 비교**: 정렬 후 데이터 값이 완전히 동일한지 확인

### 메모리 최적화
- 각 단계에서 `fu_n` 변수를 재사용하여 메모리 사용량 최소화
- 비교 완료 후 불필요한 변수 삭제


In [27]:
# fu_{n+1} == dt_year_{n+1} + fu_n 확인 (메모리 최적화: 변수 재사용)
base_dir = '/home/hashjamm/project_data/disease_network/sas_files'

comparison_results = []

# fu_1 먼저 읽기
print("=" * 80)
print("fu_1 파일 읽기 중...")
print("=" * 80)
fu_n_path = os.path.join(base_dir, 'outcome_fu_1.sas7bdat')
fu_n = pd.read_sas(fu_n_path, encoding='utf-8')
print(f"fu_1: {len(fu_n)}행, 칼럼: {list(fu_n.columns)}")

# fu_2부터 fu_10까지 확인
for n in range(1, 10):
    n_plus_1 = n + 1
    
    # dt_year_{n+1} 읽기
    dt_year_path = os.path.join(base_dir, f'outcome_dt_year_{n_plus_1}.sas7bdat')
    print(f"\n{'='*80}")
    print(f"n={n}: fu_{n_plus_1} == dt_year_{n_plus_1} + fu_{n} 확인")
    print(f"{'='*80}")
    
    try:
        dt_year_n_plus_1 = pd.read_sas(dt_year_path, encoding='utf-8')
        print(f"dt_year_{n_plus_1}: {len(dt_year_n_plus_1)}행, 칼럼: {list(dt_year_n_plus_1.columns)}")
        
        # fu_{n+1} 읽기
        fu_n_plus_1_path = os.path.join(base_dir, f'outcome_fu_{n_plus_1}.sas7bdat')
        fu_n_plus_1 = pd.read_sas(fu_n_plus_1_path, encoding='utf-8')
        print(f"fu_{n_plus_1}: {len(fu_n_plus_1)}행, 칼럼: {list(fu_n_plus_1.columns)}")
        
        # fu_n과 dt_year_{n+1} 합치기
        combined = pd.concat([fu_n, dt_year_n_plus_1], ignore_index=True)
        print(f"합친 결과: {len(combined)}행")
        
        # 칼럼 비교
        cols_fu = set(fu_n_plus_1.columns)
        cols_combined = set(combined.columns)
        cols_equal = cols_fu == cols_combined
        
        result = {
            'n': n,
            'fu_n_plus_1_rows': len(fu_n_plus_1),
            'combined_rows': len(combined),
            'rows_equal': len(fu_n_plus_1) == len(combined),
            'cols_equal': cols_equal,
            'fu_n_plus_1_cols': sorted(list(cols_fu)),
            'combined_cols': sorted(list(cols_combined))
        }
        
        # 칼럼이 같고 행 수가 같으면 값 비교
        if cols_equal and len(fu_n_plus_1) == len(combined):
            # 정렬 후 비교
            sort_cols = sorted(list(cols_fu))
            fu_sorted = fu_n_plus_1[sort_cols].sort_values(by=sort_cols).reset_index(drop=True)
            combined_sorted = combined[sort_cols].sort_values(by=sort_cols).reset_index(drop=True)
            
            values_equal = fu_sorted.equals(combined_sorted)
            result['values_equal'] = values_equal
            
            if values_equal:
                print(f"✅ fu_{n_plus_1} == dt_year_{n_plus_1} + fu_{n} (동일함)")
            else:
                print(f"❌ fu_{n_plus_1} != dt_year_{n_plus_1} + fu_{n} (값 차이 있음)")
                # 차이 확인
                diff = fu_sorted.compare(combined_sorted)
                print(f"   차이 있는 행 수: {len(diff)}")
        else:
            result['values_equal'] = None
            if not cols_equal:
                print(f"❌ 칼럼이 다름")
                print(f"   fu_{n_plus_1} 칼럼: {result['fu_n_plus_1_cols']}")
                print(f"   합친 결과 칼럼: {result['combined_cols']}")
            if len(fu_n_plus_1) != len(combined):
                print(f"❌ 행 수가 다름: fu_{n_plus_1}={len(fu_n_plus_1)}, 합친 결과={len(combined)}")
        
        comparison_results.append(result)
        
        # 메모리 최적화: fu_n을 fu_{n+1}로 덮어쓰기
        fu_n = fu_n_plus_1.copy()
        del fu_n_plus_1, dt_year_n_plus_1, combined
        
    except FileNotFoundError as e:
        print(f"❌ 파일을 찾을 수 없습니다: {e}")
        comparison_results.append({
            'n': n,
            'error': f'FileNotFoundError: {str(e)}'
        })
        break
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        comparison_results.append({
            'n': n,
            'error': str(e)
        })
        break

# 요약 출력
print(f"\n{'='*80}")
print("요약")
print(f"{'='*80}")
success_count = sum(1 for r in comparison_results if r.get('values_equal') == True)
total_count = len([r for r in comparison_results if 'error' not in r])
print(f"성공: {success_count}/{total_count}개")


fu_1 파일 읽기 중...
fu_1: 3311623행, 칼럼: ['PERSON_ID', 'abb_sick']

n=1: fu_2 == dt_year_2 + fu_1 확인
dt_year_2: 2584644행, 칼럼: ['PERSON_ID', 'abb_sick']
fu_2: 5896267행, 칼럼: ['PERSON_ID', 'abb_sick']
합친 결과: 5896267행
✅ fu_2 == dt_year_2 + fu_1 (동일함)

n=2: fu_3 == dt_year_3 + fu_2 확인
dt_year_3: 2333612행, 칼럼: ['PERSON_ID', 'abb_sick']
fu_3: 8229879행, 칼럼: ['PERSON_ID', 'abb_sick']
합친 결과: 8229879행
✅ fu_3 == dt_year_3 + fu_2 (동일함)

n=3: fu_4 == dt_year_4 + fu_3 확인
dt_year_4: 2320310행, 칼럼: ['PERSON_ID', 'abb_sick']
fu_4: 10550189행, 칼럼: ['PERSON_ID', 'abb_sick']
합친 결과: 10550189행
✅ fu_4 == dt_year_4 + fu_3 (동일함)

n=4: fu_5 == dt_year_5 + fu_4 확인
dt_year_5: 2216576행, 칼럼: ['PERSON_ID', 'abb_sick']
fu_5: 12766765행, 칼럼: ['PERSON_ID', 'abb_sick']
합친 결과: 12766765행
✅ fu_5 == dt_year_5 + fu_4 (동일함)

n=5: fu_6 == dt_year_6 + fu_5 확인
dt_year_6: 2218570행, 칼럼: ['PERSON_ID', 'abb_sick']
fu_6: 14985335행, 칼럼: ['PERSON_ID', 'abb_sick']
합친 결과: 14985335행
✅ fu_6 == dt_year_6 + fu_5 (동일함)

n=6: fu_7 == dt_year_7 + fu_6 확

## 3. std_pop3 파일 비교

두 디렉토리에 있는 `std_pop3.sas7bdat` 파일을 비교하여 칼럼과 값의 차이점을 분석합니다.

### 비교 대상
- **파일 1**: `/home/hashjamm/project_data/disease_network/sas_files/hr_project/std_pop3.sas7bdat`
- **파일 2**: `/home/hashjamm/project_data/disease_network/sas_files/std_pop3.sas7bdat`

### 비교 항목
1. **칼럼 차이**: 각 파일의 칼럼 구조 비교
   - 파일1에만 있는 칼럼
   - 파일2에만 있는 칼럼
   - 공통 칼럼

2. **값 차이**: 공통 칼럼의 데이터 값 비교
   - 비교 조건: 행 수가 동일하고 `PERSON_ID` 종류가 동일한 경우에만 수행
   - `PERSON_ID` 기준으로 정렬 후 값 비교
   - 차이가 있는 칼럼 식별


In [2]:
# std_pop3 파일 비교 (1번 항목과 동일한 로직)
file1_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/std_pop3.sas7bdat'
file2_path = '/home/hashjamm/project_data/disease_network/sas_files/std_pop3.sas7bdat'

print("=" * 80)
print("std_pop3 파일 비교")
print("=" * 80)

try:
    # 파일 읽기
    df1 = pd.read_sas(file1_path, encoding='utf-8')
    df2 = pd.read_sas(file2_path, encoding='utf-8')
    
    print(f"\n파일 1: {len(df1)}행, {len(df1.columns)}개 칼럼")
    print(f"파일 2: {len(df2)}행, {len(df2.columns)}개 칼럼")
    
    # 칼럼 차이 확인
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    only_in_file1 = cols1 - cols2
    only_in_file2 = cols2 - cols1
    common_cols = cols1 & cols2
    
    print(f"\n📋 칼럼 비교:")
    print(f"   파일1 칼럼: {sorted(list(cols1))}")
    print(f"   파일2 칼럼: {sorted(list(cols2))}")
    print(f"   공통 칼럼: {sorted(list(common_cols))}")
    
    if len(only_in_file1) > 0:
        print(f"   ⚠️ 파일1에만 있는 칼럼: {sorted(list(only_in_file1))}")
    if len(only_in_file2) > 0:
        print(f"   ⚠️ 파일2에만 있는 칼럼: {sorted(list(only_in_file2))}")
    if len(only_in_file1) == 0 and len(only_in_file2) == 0:
        print(f"   ✅ 칼럼 동일")
    
    # person_id 칼럼 확인
    person_id_col = None
    for col_name in ['PERSON_ID', 'person_id', 'Person_ID']:
        if col_name in common_cols:
            person_id_col = col_name
            break
    
    # 행 수 비교
    print(f"\n📊 행 수 비교:")
    print(f"   파일1: {len(df1)}행")
    print(f"   파일2: {len(df2)}행")
    rows_equal = len(df1) == len(df2)
    if rows_equal:
        print(f"   ✅ 행 수 동일")
    else:
        print(f"   ⚠️ 행 수 다름 (차이: {len(df1) - len(df2)}행)")
    
    # 공통 칼럼(겹치는 칼럼)만 선택하여 완전 일치 확인
    if len(common_cols) > 0:
        common_cols_list = sorted(list(common_cols))
        df1_common_only = df1[common_cols_list].copy()  # 공통 칼럼만
        df2_common_only = df2[common_cols_list].copy()  # 공통 칼럼만
        
        print(f"\n📊 공통 칼럼 완전 일치 확인:")
        print(f"   비교 대상 칼럼 ({len(common_cols_list)}개): {common_cols_list}")
        
        # 중복 제거 (공통 칼럼 기준)
        df1_unique = df1_common_only.drop_duplicates()
        df2_unique = df2_common_only.drop_duplicates()
        
        print(f"\n   파일1: 총 {len(df1_common_only)}행, 중복 제거 후 {len(df1_unique)}행")
        print(f"   파일2: 총 {len(df2_common_only)}행, 중복 제거 후 {len(df2_unique)}행")
        
        # 정렬 후 완전 일치 확인
        df1_sorted = df1_unique.sort_values(by=common_cols_list).reset_index(drop=True)
        df2_sorted = df2_unique.sort_values(by=common_cols_list).reset_index(drop=True)
        
        # 완전 일치 여부 확인
        is_identical = df1_sorted.equals(df2_sorted)
        
        if is_identical:
            print(f"\n   ✅ 공통 칼럼의 값이 완전히 동일함")
        else:
            print(f"\n   ⚠️ 공통 칼럼의 값이 다름")
            
            # 차이 분석: 어느 쪽에만 있는 행 찾기
            # merge를 사용하여 차이점 확인
            merged = df1_unique.merge(df2_unique, on=common_cols_list, how='outer', indicator=True)
            
            # 양쪽에 모두 있는 행 (겹치는 행)
            common_rows = merged[merged['_merge'] == 'both']
            # 파일1에만 있는 행
            only_in_file1 = merged[merged['_merge'] == 'left_only']
            # 파일2에만 있는 행
            only_in_file2 = merged[merged['_merge'] == 'right_only']
            
            print(f"\n   📈 차이 분석:")
            print(f"      • 겹치는 행 (양쪽 모두 존재): {len(common_rows)}개")
            print(f"      • 파일1에만 있는 행: {len(only_in_file1)}개")
            print(f"      • 파일2에만 있는 행: {len(only_in_file2)}개")
            
            # 샘플 출력
            if len(only_in_file1) > 0:
                print(f"\n   파일1에만 있는 행 샘플 (처음 5개):")
                print(only_in_file1[common_cols_list].head(5))
            if len(only_in_file2) > 0:
                print(f"\n   파일2에만 있는 행 샘플 (처음 5개):")
                print(only_in_file2[common_cols_list].head(5))
    else:
        print(f"\n📊 비교: ⚠️ 겹치는 칼럼이 없으므로 비교 불가")
    
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    traceback.print_exc()


std_pop3 파일 비교

파일 1: 2371213행, 41개 칼럼
파일 2: 2371213행, 18개 칼럼

📋 칼럼 비교:
   파일1 칼럼: ['AGE_GROUP', 'CTRB_PT_TYPE_CD', 'DFAB_GRD_CD', 'DFAB_PTN_CD', 'DFAB_REG_YM', 'DMD_CT_TOT_AMT', 'DMD_DRG_NO', 'DMD_JBRDN_AMT', 'DMD_MRI_TOT_AMT', 'DMD_SBRDN_AMT', 'DMD_TRAMT', 'DSBJT_CD', 'DTH_CODE1', 'DTH_CODE2', 'DTH_YM', 'EDEC_ADD_RT', 'EDEC_CT_TOT_AMT', 'EDEC_JBRDN_AMT', 'EDEC_MRI_TOT_AMT', 'EDEC_SBRDN_AMT', 'EDEC_TRAMT', 'FORM_CD', 'FST_IN_PAT_DT', 'IN_PAT_CORS_TYPE', 'IPSN_TYPE_CD', 'KEY_SEQ', 'MAIN_SICK', 'MPRSC_GRANT_NO', 'OFFC_INJ_TYPE', 'PERSON_ID', 'RECN', 'RECU_FR_DT', 'SEX', 'SGG', 'SIDO', 'SUB_SICK', 'TOT_PRES_DD_CNT', 'VSCN', 'YKIHO_ID', 'abb_sick', 'recu_y']
   파일2 칼럼: ['AGE_GROUP', 'CTRB_PT_TYPE_CD', 'DSBJT_CD', 'FORM_CD', 'FST_IN_PAT_DT', 'IN_PAT_CORS_TYPE', 'KEY_SEQ', 'MAIN_SICK', 'PERSON_ID', 'RECN', 'RECU_FR_DT', 'SEX', 'SGG', 'STND_Y', 'SUB_SICK', 'VSCN', 'abb_sick', 'recu_y']
   공통 칼럼: ['AGE_GROUP', 'CTRB_PT_TYPE_CD', 'DSBJT_CD', 'FORM_CD', 'FST_IN_PAT_DT', 'IN_PAT_CORS_TYPE', 'KEY

# [Issue Report] RR vs HR 프로젝트 간 데이터 정합성 분석 및 대응 전략

---

## 1. 개요 (Issue Summary)

**상황**: 기존 질병 네트워크(RR 기반, Code 1)와 후속 생존 분석(HR 기반, Code 2)의 데이터 정합성을 검증하던 중, 동일해야 할 `std_pop3` 테이블에서 일부 불일치 발견.

### 불일치 내역

- 전체 행 수(N)는 동일하나, 약 **0.17% (4,030건)**의 데이터에서 `KEY_SEQ`(청구일련번호) 등의 칼럼 값이 다름.
- `PERSON_ID`, `RECU_FR_DT`(진료개시일) 등 핵심 식별자는 동일함.

---

## 2. 원인 분석 (Root Cause Analysis)

### SAS PROC SORT의 비결정적(Non-deterministic) 특성

- 두 코드 모두 `nodupkey by person_id abb_sick`을 사용하여 중복을 제거함.
- 그러나 동일한 날짜(`RECU_FR_DT`)에 **2건 이상의 청구(다중 내원, 청구 분리 등)**가 존재하는 경우, 정렬 기준(`by`)에 `KEY_SEQ`가 포함되지 않아 무작위로 대표 행이 선택됨.

### 칼럼 구성의 차이

- Code 1은 일부 칼럼만 `KEEP`한 상태였고, Code 2는 전체 칼럼을 유지한 상태였음.
- 이로 인해 메모리 상의 정렬/선택 과정에서 미세한 차이가 발생하여, 서로 다른 청구 건(예: 외래 vs 처방)이 선택됨.

---

## 3. 영향 평가 (Impact Assessment)

### 통계적 유의성 (✅ 안전)

- `matched` 코호트(모집단)가 양쪽 프로젝트에서 완벽히 동일함을 확인.
- 불일치한 0.17%의 경우도 **"해당 일자에 질병이 발생했다"**는 사실(Event Occurrence) 자체는 변함이 없음.
- 따라서 RR(Risk Ratio) 및 HR(Hazard Ratio)의 통계적 수치에는 영향이 없음.

### 데이터 무결성 (⚠️ 주의)

- RR 분석 결과에는 `KEY_SEQ` 정보가 누락되어 있거나 HR과 다른 건을 가리킬 수 있음.
- 시각화 및 DB 구축 시 Node와 Edge를 구체적인 청구 건과 매핑할 때 정리가 필요함.

In [5]:
# outcome_dt_year_10 파일 비교 (std_pop3 비교와 동일한 로직)
file1_path = '/home/hashjamm/project_data/disease_network/sas_files/outcome_dt_year_10.sas7bdat'
file2_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_10.sas7bdat'

print("=" * 80)
print("outcome_dt_year_10 파일 비교")
print("=" * 80)

try:
    # 파일 읽기
    df1 = pd.read_sas(file1_path, encoding='utf-8')
    df2 = pd.read_sas(file2_path, encoding='utf-8')
    
    print(f"\n파일 1: {len(df1)}행, {len(df1.columns)}개 칼럼")
    print(f"파일 2: {len(df2)}행, {len(df2.columns)}개 칼럼")
    
    # 칼럼 차이 확인
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    only_in_file1 = cols1 - cols2
    only_in_file2 = cols2 - cols1
    common_cols = cols1 & cols2
    
    print(f"\n📋 칼럼 비교:")
    print(f"   파일1 칼럼: {sorted(list(cols1))}")
    print(f"   파일2 칼럼: {sorted(list(cols2))}")
    print(f"   공통 칼럼: {sorted(list(common_cols))}")
    
    if len(only_in_file1) > 0:
        print(f"   ⚠️ 파일1에만 있는 칼럼: {sorted(list(only_in_file1))}")
    if len(only_in_file2) > 0:
        print(f"   ⚠️ 파일2에만 있는 칼럼: {sorted(list(only_in_file2))}")
    if len(only_in_file1) == 0 and len(only_in_file2) == 0:
        print(f"   ✅ 칼럼 동일")
    
    # person_id 칼럼 확인
    person_id_col = None
    for col_name in ['PERSON_ID', 'person_id', 'Person_ID']:
        if col_name in common_cols:
            person_id_col = col_name
            break
    
    # 행 수 비교
    print(f"\n📊 행 수 비교:")
    print(f"   파일1: {len(df1)}행")
    print(f"   파일2: {len(df2)}행")
    rows_equal = len(df1) == len(df2)
    if rows_equal:
        print(f"   ✅ 행 수 동일")
    else:
        print(f"   ⚠️ 행 수 다름 (차이: {len(df1) - len(df2)}행)")
    
    # 공통 칼럼(겹치는 칼럼)만 선택하여 완전 일치 확인
    if len(common_cols) > 0:
        common_cols_list = sorted(list(common_cols))
        df1_common_only = df1[common_cols_list].copy()  # 공통 칼럼만
        df2_common_only = df2[common_cols_list].copy()  # 공통 칼럼만
        
        print(f"\n📊 공통 칼럼 완전 일치 확인:")
        print(f"   비교 대상 칼럼 ({len(common_cols_list)}개): {common_cols_list}")
        
        # 중복 제거 (공통 칼럼 기준)
        df1_unique = df1_common_only.drop_duplicates()
        df2_unique = df2_common_only.drop_duplicates()
        
        print(f"\n   파일1: 총 {len(df1_common_only)}행, 중복 제거 후 {len(df1_unique)}행")
        print(f"   파일2: 총 {len(df2_common_only)}행, 중복 제거 후 {len(df2_unique)}행")
        
        # 정렬 후 완전 일치 확인
        df1_sorted = df1_unique.sort_values(by=common_cols_list).reset_index(drop=True)
        df2_sorted = df2_unique.sort_values(by=common_cols_list).reset_index(drop=True)
        
        # 완전 일치 여부 확인
        is_identical = df1_sorted.equals(df2_sorted)
        
        if is_identical:
            print(f"\n   ✅ 공통 칼럼의 값이 완전히 동일함")
        else:
            print(f"\n   ⚠️ 공통 칼럼의 값이 다름")
            
            # 차이 분석: 어느 쪽에만 있는 행 찾기
            # merge를 사용하여 차이점 확인
            merged = df1_unique.merge(df2_unique, on=common_cols_list, how='outer', indicator=True)
            
            # 양쪽에 모두 있는 행 (겹치는 행)
            common_rows = merged[merged['_merge'] == 'both']
            # 파일1에만 있는 행
            only_in_file1 = merged[merged['_merge'] == 'left_only']
            # 파일2에만 있는 행
            only_in_file2 = merged[merged['_merge'] == 'right_only']
            
            print(f"\n   📈 차이 분석:")
            print(f"      • 겹치는 행 (양쪽 모두 존재): {len(common_rows)}개")
            print(f"      • 파일1에만 있는 행: {len(only_in_file1)}개")
            print(f"      • 파일2에만 있는 행: {len(only_in_file2)}개")
            
            # 샘플 출력
            if len(only_in_file1) > 0:
                print(f"\n   파일1에만 있는 행 샘플 (처음 5개):")
                print(only_in_file1[common_cols_list].head(5))
            if len(only_in_file2) > 0:
                print(f"\n   파일2에만 있는 행 샘플 (처음 5개):")
                print(only_in_file2[common_cols_list].head(5))
    else:
        print(f"\n📊 비교: ⚠️ 겹치는 칼럼이 없으므로 비교 불가")
    
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    traceback.print_exc()


outcome_dt_year_10 파일 비교

파일 1: 1968727행, 2개 칼럼
파일 2: 1968727행, 3개 칼럼

📋 칼럼 비교:
   파일1 칼럼: ['PERSON_ID', 'abb_sick']
   파일2 칼럼: ['KEY_SEQ', 'PERSON_ID', 'abb_sick']
   공통 칼럼: ['PERSON_ID', 'abb_sick']
   ⚠️ 파일2에만 있는 칼럼: ['KEY_SEQ']

📊 행 수 비교:
   파일1: 1968727행
   파일2: 1968727행
   ✅ 행 수 동일

📊 공통 칼럼 완전 일치 확인:
   비교 대상 칼럼 (2개): ['PERSON_ID', 'abb_sick']

   파일1: 총 1968727행, 중복 제거 후 1968727행
   파일2: 총 1968727행, 중복 제거 후 1968727행

   ✅ 공통 칼럼의 값이 완전히 동일함


In [7]:
# outcome_fu_10 파일 비교 (std_pop3 비교와 동일한 로직)
file1_path = '/home/hashjamm/project_data/disease_network/sas_files/outcome_fu_10.sas7bdat'
file2_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_fu_10.sas7bdat'

print("=" * 80)
print("outcome_fu_10 파일 비교")
print("=" * 80)

try:
    # 파일 읽기
    df1 = pd.read_sas(file1_path, encoding='utf-8')
    df2 = pd.read_sas(file2_path, encoding='utf-8')
    
    print(f"\n파일 1: {len(df1)}행, {len(df1.columns)}개 칼럼")
    print(f"파일 2: {len(df2)}행, {len(df2.columns)}개 칼럼")
    
    # 칼럼 차이 확인
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    only_in_file1 = cols1 - cols2
    only_in_file2 = cols2 - cols1
    common_cols = cols1 & cols2
    
    print(f"\n📋 칼럼 비교:")
    print(f"   파일1 칼럼: {sorted(list(cols1))}")
    print(f"   파일2 칼럼: {sorted(list(cols2))}")
    print(f"   공통 칼럼: {sorted(list(common_cols))}")
    
    if len(only_in_file1) > 0:
        print(f"   ⚠️ 파일1에만 있는 칼럼: {sorted(list(only_in_file1))}")
    if len(only_in_file2) > 0:
        print(f"   ⚠️ 파일2에만 있는 칼럼: {sorted(list(only_in_file2))}")
    if len(only_in_file1) == 0 and len(only_in_file2) == 0:
        print(f"   ✅ 칼럼 동일")
    
    # person_id 칼럼 확인
    person_id_col = None
    for col_name in ['PERSON_ID', 'person_id', 'Person_ID']:
        if col_name in common_cols:
            person_id_col = col_name
            break
    
    # 행 수 비교
    print(f"\n📊 행 수 비교:")
    print(f"   파일1: {len(df1)}행")
    print(f"   파일2: {len(df2)}행")
    rows_equal = len(df1) == len(df2)
    if rows_equal:
        print(f"   ✅ 행 수 동일")
    else:
        print(f"   ⚠️ 행 수 다름 (차이: {len(df1) - len(df2)}행)")
    
    # 공통 칼럼(겹치는 칼럼)만 선택하여 완전 일치 확인
    if len(common_cols) > 0:
        common_cols_list = sorted(list(common_cols))
        df1_common_only = df1[common_cols_list].copy()  # 공통 칼럼만
        df2_common_only = df2[common_cols_list].copy()  # 공통 칼럼만
        
        print(f"\n📊 공통 칼럼 완전 일치 확인:")
        print(f"   비교 대상 칼럼 ({len(common_cols_list)}개): {common_cols_list}")
        
        # 중복 제거 (공통 칼럼 기준)
        df1_unique = df1_common_only.drop_duplicates()
        df2_unique = df2_common_only.drop_duplicates()
        
        print(f"\n   파일1: 총 {len(df1_common_only)}행, 중복 제거 후 {len(df1_unique)}행")
        print(f"   파일2: 총 {len(df2_common_only)}행, 중복 제거 후 {len(df2_unique)}행")
        
        # 정렬 후 완전 일치 확인
        df1_sorted = df1_unique.sort_values(by=common_cols_list).reset_index(drop=True)
        df2_sorted = df2_unique.sort_values(by=common_cols_list).reset_index(drop=True)
        
        # 완전 일치 여부 확인
        is_identical = df1_sorted.equals(df2_sorted)
        
        if is_identical:
            print(f"\n   ✅ 공통 칼럼의 값이 완전히 동일함")
        else:
            print(f"\n   ⚠️ 공통 칼럼의 값이 다름")
            
            # 차이 분석: 어느 쪽에만 있는 행 찾기
            # merge를 사용하여 차이점 확인
            merged = df1_unique.merge(df2_unique, on=common_cols_list, how='outer', indicator=True)
            
            # 양쪽에 모두 있는 행 (겹치는 행)
            common_rows = merged[merged['_merge'] == 'both']
            # 파일1에만 있는 행
            only_in_file1 = merged[merged['_merge'] == 'left_only']
            # 파일2에만 있는 행
            only_in_file2 = merged[merged['_merge'] == 'right_only']
            
            print(f"\n   📈 차이 분석:")
            print(f"      • 겹치는 행 (양쪽 모두 존재): {len(common_rows)}개")
            print(f"      • 파일1에만 있는 행: {len(only_in_file1)}개")
            print(f"      • 파일2에만 있는 행: {len(only_in_file2)}개")
            
            # 샘플 출력
            if len(only_in_file1) > 0:
                print(f"\n   파일1에만 있는 행 샘플 (처음 5개):")
                print(only_in_file1[common_cols_list].head(5))
            if len(only_in_file2) > 0:
                print(f"\n   파일2에만 있는 행 샘플 (처음 5개):")
                print(only_in_file2[common_cols_list].head(5))
    else:
        print(f"\n📊 비교: ⚠️ 겹치는 칼럼이 없으므로 비교 불가")
    
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    traceback.print_exc()


outcome_fu_10 파일 비교

파일 1: 23319782행, 2개 칼럼
파일 2: 23319782행, 3개 칼럼

📋 칼럼 비교:
   파일1 칼럼: ['PERSON_ID', 'abb_sick']
   파일2 칼럼: ['KEY_SEQ', 'PERSON_ID', 'abb_sick']
   공통 칼럼: ['PERSON_ID', 'abb_sick']
   ⚠️ 파일2에만 있는 칼럼: ['KEY_SEQ']

📊 행 수 비교:
   파일1: 23319782행
   파일2: 23319782행
   ✅ 행 수 동일

📊 공통 칼럼 완전 일치 확인:
   비교 대상 칼럼 (2개): ['PERSON_ID', 'abb_sick']

   파일1: 총 23319782행, 중복 제거 후 23319782행
   파일2: 총 23319782행, 중복 제거 후 23319782행

   ✅ 공통 칼럼의 값이 완전히 동일함


## 4. outcome_dt 및 outcome_fu 테이블 구조 분석 및 결론

### 데이터 구조 확인 결과

**RR 프로젝트의 핵심 테이블 구조**:
- `outcome_dt_year_{n}`: `PERSON_ID`, `abb_sick` (2개 칼럼)
- `outcome_fu_{n}`: `PERSON_ID`, `abb_sick` (2개 칼럼)

**HR 프로젝트의 새로운 테이블 구조**:
- `outcome_dt_year_{n}`: `KEY_SEQ`, `PERSON_ID`, `abb_sick` (3개 칼럼)
- `outcome_fu_{n}`: `KEY_SEQ`, `PERSON_ID`, `abb_sick` (3개 칼럼)

### 핵심 발견 사항

1. **공통 칼럼의 완전 일치**: 
   - `PERSON_ID`와 `abb_sick` 기준으로 두 프로젝트의 데이터가 **100% 동일**함을 확인
   - 행 수도 완전히 동일함

2. **KEY_SEQ의 역할**:
   - HR 버전에만 `KEY_SEQ` 칼럼이 추가되어 있음
   - 이는 동일한 사람(`PERSON_ID`), 동일한 질병(`abb_sick`), 동일한 일자에 대해 여러 청구 건이 존재할 수 있음을 의미

### SAS nodupkey 로직의 영향

**SAS `PROC SORT nodupkey`의 비결정적 특성**:
- 정렬 기준(`by`)에 포함된 칼럼 수에 따라 중복 제거 시 선택되는 행이 달라질 수 있음
- RR 버전: `by person_id abb_sick` (2개 칼럼 기준)
- HR 버전: `by person_id abb_sick key_seq` (3개 칼럼 기준, 또는 다른 칼럼 포함)

**결과적으로**:
- 같은 사람, 같은 질병, 같은 일자에 대해 **서로 다른 `KEY_SEQ`가 부여될 수 있음**
- 그러나 `PERSON_ID`와 `abb_sick`의 조합은 완전히 동일하므로, **질병 발생 사건(Event) 자체는 변하지 않음**

---

## 5. 최종 결론 및 대응 전략

### ✅ RR 통계 수치의 무결성 보장

- **RR 통계 계산 결과는 그대로 사용 가능**
  - `PERSON_ID`와 `abb_sick` 기준으로 데이터가 완전히 동일함
  - 질병 발생 사건의 빈도와 패턴이 동일하므로 Risk Ratio 계산에 영향 없음

### ✅ edge_pids 데이터의 재사용 가능

- **기존에 구해둔 `edge_pids` (Edge가 형성된 PID 리스트)는 그대로 사용 가능**
  - Edge 형성은 `PERSON_ID`와 `abb_sick` 조합을 기반으로 하므로
  - 네트워크 구조(Topology)는 변하지 않음

### ⚠️ Edge의 KEY 값 매핑 전략

- **Edge에 부여되는 `key_seq` 값은 새로운 HR 버전의 `outcome_dt` 및 `outcome_fu` 테이블을 따라야 함**
  - RR 분석 시점에는 `KEY_SEQ` 정보가 없었거나 사용할 수 없었음
  - 동일한 `(PERSON_ID, abb_sick, 날짜)` 조합에 대해 다른 `KEY_SEQ`가 선택될 수 있지만, 이는 네트워크 구조에는 영향 없음
  - Edge의 `key_seq` 매핑은 HR 프로젝트의 `outcome_dt_year` 및 `outcome_fu` 테이블을 참조하여 수행

### 구현 방향

1. **RR 통계 및 edge_pids**: 기존 결과 그대로 유지 및 사용
2. **Edge의 key_seq 매핑**: HR 프로젝트의 `outcome_dt_year` 및 `outcome_fu` 테이블을 Lookup 테이블로 사용하여 매핑
3. **데이터 무결성**: 통계적 수치는 변하지 않으며, 단지 Edge에 대한 메타데이터(`key_seq`)만 새로 매핑하면 됨

---

**요약**: RR의 통계 수치와 네트워크 구조는 완전히 무결하게 유지되며, Edge의 `key_seq` 값만 HR 버전의 outcome 테이블을 기준으로 새로 매핑하면 됨.


In [ ]:

file_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_10.pkl'

# pickle 파일 로드
with open(file_path, 'rb') as f:
    edge_pids_10 = pickle.load(f)

# 데이터 타입 확인
print(f"데이터 타입: {type(edge_pids_10)}")

# 데이터 구조 확인
if isinstance(edge_pids_10, dict):
    print(f"\n딕셔너리 키 개수: {len(edge_pids_10)}")
    print(f"키 목록 (처음 10개): {list(edge_pids_10.keys())[:10]}")
    
    # 첫 번째 키의 값 확인
    if len(edge_pids_10) > 0:
        first_key = list(edge_pids_10.keys())[0]
        first_value = edge_pids_10[first_key]
        print(f"\n첫 번째 키: {first_key}")
        print(f"첫 번째 값 타입: {type(first_value)}")
        if isinstance(first_value, (list, set, tuple)):
            print(f"첫 번째 값 길이: {len(first_value)}")
            print(f"첫 번째 값 샘플 (처음 10개): {list(first_value)[:10]}")
        elif isinstance(first_value, dict):
            print(f"첫 번째 값 키 개수: {len(first_value)}")
            print(f"첫 번째 값 키 샘플: {list(first_value.keys())[:10]}")
        else:
            print(f"첫 번째 값: {first_value}")
    
    # 전체 딕셔너리 샘플 (처음 5개 항목)
    print(f"\n전체 데이터 샘플 (처음 5개 항목):")
    for i, (key, value) in enumerate(list(edge_pids_10.items())[:5]):
        if isinstance(value, (list, set, tuple)):
            print(f"  {key}: {type(value).__name__} (길이: {len(value)}, 샘플: {list(value)[:5]})")
        elif isinstance(value, dict):
            print(f"  {key}: dict (키 개수: {len(value)}, 키 샘플: {list(value.keys())[:5]})")
        else:
            print(f"  {key}: {value}")

elif isinstance(edge_pids_10, (list, tuple)):
    print(f"\n리스트/튜플 길이: {len(edge_pids_10)}")
    print(f"처음 10개 항목:")
    for i, item in enumerate(edge_pids_10[:10]):
        print(f"  [{i}]: {item}")

elif isinstance(edge_pids_10, set):
    print(f"\n셋 크기: {len(edge_pids_10)}")
    print(f"처음 10개 항목:")
    for i, item in enumerate(list(edge_pids_10)[:10]):
        print(f"  [{i}]: {item}")

else:
    print(f"\n데이터: {edge_pids_10}")
    print(f"데이터 크기/길이: {len(edge_pids_10) if hasattr(edge_pids_10, '__len__') else 'N/A'}")


데이터 타입: <class 'dict'>

딕셔너리 키 개수: 320428
키 목록 (처음 10개): [('J03', 'J20'), ('J03', 'J06'), ('J03', 'J02'), ('J03', 'J00'), ('J03', 'K29'), ('J03', 'H10'), ('J03', 'L23'), ('J03', 'J30'), ('J03', 'J04'), ('J03', 'J01')]

첫 번째 키: ('J03', 'J20')
첫 번째 값 타입: <class 'numpy.ndarray'>
첫 번째 값: [10115216 10370629 10377237 ... 95717664 95755730 95820119]

전체 데이터 샘플 (처음 5개 항목):
  ('J03', 'J20'): [10115216 10370629 10377237 ... 95717664 95755730 95820119]
  ('J03', 'J06'): [10062980 10119030 10261227 ... 95613138 95707652 95822728]
  ('J03', 'J02'): [10351340 10424308 10446492 ... 95708666 95788401 95819070]
  ('J03', 'J00'): [10119030 10140151 10559362 ... 95630245 95698512 95761924]
  ('J03', 'K29'): [10232145 10300910 10402504 ... 95766801 95788401 95806862]


In [11]:
# DBver_edge_stat.csv 파일 읽기 및 상위 5개 행 확인
file_path = '/home/hashjamm/results/disease_network/db_migration/DBver_edge_stat.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 데이터프레임 정보 확인
print(f"데이터프레임 shape: {df.shape}")
print(f"\n칼럼 목록:\n{df.columns.tolist()}")
print(f"\n상위 5개 행:")
print(df.head(5))

# fu가 10인 경우의 행 개수 확인
fu_10_count = len(df[df['fu'] == 10])
print(f"\nfu가 10인 행 개수: {fu_10_count}")
print(f"전체 행 대비 비율: {fu_10_count / len(df) * 100:.2f}%")


데이터프레임 shape: (991885, 15)

칼럼 목록:
['fu', 'cause_abb', 'outcome_abb', 'ct00', 'ct01', 'ct10', 'ct11', 'rr_values', 'rr_lower_cis', 'rr_upper_cis', 'log_rr_values', 'chisq_p_values', 'fisher_p_values', 'adjusted_chisq_p_values', 'adjusted_fisher_p_values']

상위 5개 행:
   fu cause_abb outcome_abb  ct00  ct01  ct10  ct11  rr_values  rr_lower_cis  \
0   1       A16         A15  7061     9  1280   134  74.444444     37.999309   
1   1       K74         C22  4409     6   811    72  60.000000     26.171216   
2   1       A15         A16  5072    13   897   120  46.153846     26.145321   
3   1       G81         I69  1649     6   284    47  39.166667     16.884047   
4   1       C22         K74  2085    10   344    75  37.500000     19.548704   

   rr_upper_cis  log_rr_values  chisq_p_values  fisher_p_values  \
0    145.844107       4.310053   3.432395e-137     2.755249e-94   
1    137.555703       4.094345    6.712756e-73     7.373630e-50   
2     81.474520       3.831980   3.292205e-117     3

## Edge 데이터 구조 분석 및 정리

### 1. R 코드 (hr_calculator_engine_v3.R)에서의 Edge 매핑 데이터

R 코드에서 생성되는 Edge 매핑 데이터는 다음과 같이 구성됩니다:

```r
# Mapping 데이터
key <- paste(cause_abb, outcome_abb, fu, sep = "_")
pids <- clean_data[case == 1, .(person_id)]
idx_key <- clean_data[case == 1, .(index_key_seq)]
out_key <- clean_data[case == 1 & status == 1, .(key_seq)]
```

**각 변수의 의미:**

1. **`edge_pids`** (`pids`):
   - `case == 1`인 모든 사람들의 `person_id`
   - 원인 질병을 가진 사람들 (case group) 전체
   - 결과 질병 발생 여부와 무관하게 포함됨
   - **실제로는 Edge가 아니라 원인 질병 노드에 속한 사람들의 정보**

2. **`edge_index_key_seq`** (`idx_key`):
   - `case == 1`인 모든 사람들의 `index_key_seq`
   - 원인 질병 발생 시점의 키
   - 결과 질병 발생 여부와 무관하게 포함됨
   - **역시 Edge가 아니라 원인 질병 노드의 정보**

3. **`edge_key_seq`** (`out_key`):
   - `case == 1 & status == 1`인 사람들의 `key_seq`
   - 원인 질병을 가지고 결과 질병도 발생한 사람들의 결과 질병 발생 시점 키
   - **진짜 Edge를 대표하는 데이터**

**결론:**
- 결과 질병이 발생한 사람에 대한 데이터는 `out_key`만이 맞습니다.
- `pids`와 `idx_key`는 원인 질병을 가진 전체 집단의 정보이며, Edge가 아닙니다.

---

### 2. Python 코드 (disease_network_funs.py)에서의 edge_pids 추출 방식

Python 코드의 `process_disease_pair_unfiltered` 함수에서 `edge_pids`는 다음과 같이 추출됩니다:

```python
edge_pids = one_final_np[(one_final_np[:, 1] == 1) & (one_final_np[:, 2] == 1)][:, 0]
```

**추출 과정:**

1. **데이터 준비:**
   - `cause_np`: `[PERSON_ID, cause]` 형태 (matched 데이터)
   - `outcome_np`: 결과 질병을 가진 사람들의 `PERSON_ID`

2. **교집합 찾기:**
   - 원인 질병 데이터와 결과 질병 데이터의 `PERSON_ID` 교집합 (`match_ids`)

3. **병합 테이블 생성:**
   - `one_final_np`: `[PERSON_ID, cause, outcome_status]` 형태
   - `cause_np`의 모든 행을 복사 (LEFT JOIN과 유사)
   - `match_ids`에 해당하는 경우에만 `outcome_status = 1`로 설정

4. **edge_pids 추출:**
   - `cause == 1` (원인 질병을 가진 사람) **AND**
   - `outcome_status == 1` (결과 질병도 발생한 사람)
   - 조건을 만족하는 `PERSON_ID`만 추출

**결론:**
- Python 코드의 `edge_pids`는 R 코드의 `out_key`와 동일한 로직입니다.
- 원인 질병을 가진 사람 중에서 결과 질병도 발생한 사람들만 포함합니다.

---

### 3. edge_pids_n의 시간 범위 의미 (중요!)

**데이터 구조:**
- `outcome_fu_{n+1}` == `outcome_dt_year_{n+1}` + `outcome_fu_n`
- 예: `outcome_fu_2` == `outcome_dt_year_2` + `outcome_fu_1`
- 즉, `outcome_dt_year_2`는 **fu=1에서 fu=2로 넘어가는 시점에 새로 발생한 질병들만** 포함합니다.

**코드 흐름:**
1. `ctable_2`는 `outcome_dt_year_2`를 사용하여 생성됩니다.
2. `edge_pids_2`도 `outcome_dt_year_2`를 사용하여 생성됩니다.
3. `updating_disease_pair` 함수는:
   ```python
   full_ctable_2 = fun_py.updating_disease_pair(full_ctable_1.to_numpy(), ctable_2.to_numpy())
   ```
   - `full_ctable_1` (fu=1까지의 누적 결과)에
   - `ctable_2` (fu=1→fu=2 사이에 새로 발생한 엣지들)를 더하여
   - `full_ctable_2` (fu=2까지의 누적 결과)를 만듭니다.

**결론:**
- **`edge_pids_2`는 fu=2 전체가 아니라, fu=1에서 fu=2로 넘어가는 시점에 새로 발생한 엣지들에 대한 pids만 포함합니다.**
- 즉, `full_ctable_1`에 추가되는 새로운 엣지들에 대한 정보입니다.
- 따라서 **fu=2 전체의 엣지를 보려면 `edge_pids_1`과 `edge_pids_2`를 합쳐야 합니다.**

**일반화:**
- `edge_pids_n` (n ≥ 2)는 fu=(n-1)에서 fu=n으로 넘어가는 시점에 새로 발생한 엣지들만 포함합니다.
- fu=n 전체의 엣지를 보려면 `edge_pids_1`부터 `edge_pids_n`까지 모두 합쳐야 합니다.


In [12]:
# 각 년도별 CSV 파일의 조합 개수와 edge_pids의 조합 개수 비교

# 경로 설정
final_results_dir = '/home/hashjamm/results/disease_network/final_results'
edge_pids_dir = '/home/hashjamm/results/disease_network/edge_pids'

# 처리할 n 값들 (1부터 10까지)
n_values = list(range(1, 11))

# 비교 결과 저장
comparison_results = {}

print("=" * 80)
print("CSV 파일 조합 개수 vs edge_pids 조합 개수 비교")
print("=" * 80)

for n in n_values:
    print(f"\n{'='*80}")
    print(f"비교 중: n = {n}")
    print(f"{'='*80}")
    
    # CSV 파일 경로
    csv_path = os.path.join(final_results_dir, f'DBver_cis_cut_final_result_{n}.csv')
    
    # pickle 파일 경로
    pkl_path = os.path.join(edge_pids_dir, f'edge_pids_{n}.pkl')
    
    # 파일 존재 확인
    if not os.path.exists(csv_path):
        print(f"⚠️ CSV 파일이 존재하지 않습니다: {csv_path}")
        continue
    
    if not os.path.exists(pkl_path):
        print(f"⚠️ Pickle 파일이 존재하지 않습니다: {pkl_path}")
        continue
    
    # CSV 파일 읽기
    print(f"CSV 파일 읽기: {csv_path}")
    df_final = pd.read_csv(csv_path)
    
    # cause_abb, outcome_abb 조합 추출
    csv_combinations = set(zip(df_final['cause_abb'], df_final['outcome_abb']))
    csv_count = len(csv_combinations)
    print(f"CSV에 있는 조합 개수: {csv_count:,}개")
    
    # pickle 파일 로드
    print(f"Pickle 파일 로드: {pkl_path}")
    with open(pkl_path, 'rb') as f:
        edge_pids_n = pickle.load(f)
    
    edge_pids_count = len(edge_pids_n)
    print(f"edge_pids_{n}의 전체 조합 개수: {edge_pids_count:,}개")
    
    # 조합 비교
    edge_pids_combos = set(edge_pids_n.keys())
    
    # 교집합, 차집합 계산
    common_combos = csv_combinations & edge_pids_combos
    only_in_csv = csv_combinations - edge_pids_combos
    only_in_edge_pids = edge_pids_combos - csv_combinations
    
    print(f"\n비교 결과:")
    print(f"  공통 조합: {len(common_combos):,}개")
    print(f"  CSV에만 있는 조합: {len(only_in_csv):,}개")
    print(f"  edge_pids에만 있는 조합: {len(only_in_edge_pids):,}개")
    
    # 결과 저장
    comparison_results[n] = {
        'csv_count': csv_count,
        'edge_pids_count': edge_pids_count,
        'common_count': len(common_combos),
        'only_in_csv': len(only_in_csv),
        'only_in_edge_pids': len(only_in_edge_pids)
    }

print(f"\n{'='*80}")
print("비교 완료!")
print(f"{'='*80}\n")

# 최종 요약 테이블
print("비교 결과 요약:")
print("-" * 100)
print(f"{'n':<5} | {'CSV 조합':<12} | {'edge_pids 조합':<15} | {'공통':<10} | {'CSV만':<10} | {'edge_pids만':<15}")
print("-" * 100)
for n in sorted(comparison_results.keys()):
    result = comparison_results[n]
    print(f"{n:<5} | {result['csv_count']:>10,}개 | {result['edge_pids_count']:>13,}개 | "
          f"{result['common_count']:>8,}개 | {result['only_in_csv']:>8,}개 | {result['only_in_edge_pids']:>13,}개")
print("-" * 100)


CSV 파일 조합 개수 vs edge_pids 조합 개수 비교

비교 중: n = 1
CSV 파일 읽기: /home/hashjamm/results/disease_network/final_results/DBver_cis_cut_final_result_1.csv
CSV에 있는 조합 개수: 55,785개
Pickle 파일 로드: /home/hashjamm/results/disease_network/edge_pids/edge_pids_1.pkl
edge_pids_1의 전체 조합 개수: 367,932개

비교 결과:
  공통 조합: 55,785개
  CSV에만 있는 조합: 0개
  edge_pids에만 있는 조합: 312,147개

비교 중: n = 2
CSV 파일 읽기: /home/hashjamm/results/disease_network/final_results/DBver_cis_cut_final_result_2.csv
CSV에 있는 조합 개수: 74,165개
Pickle 파일 로드: /home/hashjamm/results/disease_network/edge_pids/edge_pids_2.pkl
edge_pids_2의 전체 조합 개수: 341,239개

비교 결과:
  공통 조합: 73,801개
  CSV에만 있는 조합: 364개
  edge_pids에만 있는 조합: 267,438개

비교 중: n = 3
CSV 파일 읽기: /home/hashjamm/results/disease_network/final_results/DBver_cis_cut_final_result_3.csv
CSV에 있는 조합 개수: 85,354개
Pickle 파일 로드: /home/hashjamm/results/disease_network/edge_pids/edge_pids_3.pkl
edge_pids_3의 전체 조합 개수: 332,967개

비교 결과:
  공통 조합: 83,854개
  CSV에만 있는 조합: 1,500개
  edge_pids에만 있는 조합: 249,113개

비교 중: n

In [13]:
# 가설 검증: edge_pids_n이 누적 데이터인지 확인
# 가설: edge_pids_n (n ≥ 2)는 fu=(n-1)에서 fu=n으로 넘어가는 시점에 새로 발생한 엣지들만 포함
# 따라서 fu=n 전체의 엣지를 보려면 edge_pids_1부터 edge_pids_n까지 합쳐야 함
# 검증 방법: 누적 edge_pids (1부터 n까지 합친 것)의 조합과 CSV 파일의 조합을 비교

# 경로 설정
final_results_dir = '/home/hashjamm/results/disease_network/final_results'
edge_pids_dir = '/home/hashjamm/results/disease_network/edge_pids'

# 처리할 n 값들 (1부터 10까지)
n_values = list(range(1, 11))

# 검증 결과 저장
verification_results = {}

print("=" * 80)
print("가설 검증: edge_pids_n의 누적 특성 확인")
print("=" * 80)
print("\n가설: edge_pids_n (n ≥ 2)는 fu=(n-1)에서 fu=n으로 넘어가는 시점에")
print("      새로 발생한 엣지들만 포함하므로, fu=n 전체를 보려면")
print("      edge_pids_1부터 edge_pids_n까지 합쳐야 함")
print("\n검증 방법: 누적 edge_pids (1부터 n까지 합친 것)의 조합과 CSV 파일의 조합 비교\n")

for n in n_values:
    print(f"\n{'='*80}")
    print(f"검증 중: n = {n}")
    print(f"{'='*80}")
    
    # 1. CSV 파일에서 조합 읽기
    csv_path = os.path.join(final_results_dir, f'DBver_cis_cut_final_result_{n}.csv')
    
    if not os.path.exists(csv_path):
        print(f"⚠️ CSV 파일이 존재하지 않습니다: {csv_path}")
        continue
    
    df_final = pd.read_csv(csv_path)
    csv_combinations = set(zip(df_final['cause_abb'], df_final['outcome_abb']))
    csv_count = len(csv_combinations)
    print(f"CSV 파일의 조합 개수: {csv_count:,}개")
    
    # 2. edge_pids_1부터 edge_pids_n까지 로드하여 누적
    cumulative_edge_pids_combos = set()
    
    for i in range(1, n + 1):
        pkl_path = os.path.join(edge_pids_dir, f'edge_pids_{i}.pkl')
        
        if not os.path.exists(pkl_path):
            print(f"⚠️ 파일이 존재하지 않습니다: {pkl_path}")
            continue
        
        with open(pkl_path, 'rb') as f:
            edge_pids_i = pickle.load(f)
        
        # edge_pids_i의 모든 조합을 누적
        cumulative_edge_pids_combos.update(edge_pids_i.keys())
    
    cumulative_count = len(cumulative_edge_pids_combos)
    print(f"누적 edge_pids (1부터 {n}까지 합친 결과)의 조합 개수: {cumulative_count:,}개")
    
    # 3. 조합 비교
    common_combos = csv_combinations & cumulative_edge_pids_combos
    only_in_csv = csv_combinations - cumulative_edge_pids_combos
    only_in_cumulative = cumulative_edge_pids_combos - csv_combinations
    
    print(f"\n비교 결과:")
    print(f"  공통 조합: {len(common_combos):,}개")
    print(f"  CSV에만 있는 조합: {len(only_in_csv):,}개")
    print(f"  누적 edge_pids에만 있는 조합: {len(only_in_cumulative):,}개")
    
    # 검증 결과 저장
    verification_results[n] = {
        'csv_count': csv_count,
        'cumulative_count': cumulative_count,
        'common_count': len(common_combos),
        'only_in_csv': len(only_in_csv),
        'only_in_cumulative': len(only_in_cumulative),
        'is_valid': len(only_in_csv) == 0  # CSV의 모든 조합이 누적에 있어야 함
    }

print(f"\n{'='*80}")
print("검증 완료!")
print(f"{'='*80}\n")

# 최종 요약
print("검증 결과 요약:")
print("-" * 100)
print(f"{'n':<5} | {'CSV 조합':<12} | {'누적 조합':<15} | {'공통':<10} | {'CSV만':<10} | {'누적만':<15} | {'결과':<10}")
print("-" * 100)
for n in sorted(verification_results.keys()):
    result = verification_results[n]
    status = "✅ 통과" if result['is_valid'] else "❌ 실패"
    print(f"{n:<5} | {result['csv_count']:>10,}개 | {result['cumulative_count']:>13,}개 | "
          f"{result['common_count']:>8,}개 | {result['only_in_csv']:>8,}개 | "
          f"{result['only_in_cumulative']:>13,}개 | {status}")
print("-" * 100)

# 전체 통과 여부
all_valid = all(r['is_valid'] for r in verification_results.values())
if all_valid:
    print("\n✅ 가설이 검증되었습니다!")
    print("   CSV 파일의 모든 조합이 누적 edge_pids에 포함되어 있습니다.")
    print("   즉, edge_pids_n (n ≥ 2)는 fu=(n-1)에서 fu=n으로 넘어가는 시점에")
    print("   새로 발생한 엣지들만 포함하며, fu=n 전체를 보려면")
    print("   edge_pids_1부터 edge_pids_n까지 합쳐야 합니다.")
else:
    print("\n⚠️ 일부 검증 실패가 있습니다.")
    print("   CSV에만 있는 조합이 있다는 것은 누적 edge_pids에 해당 조합이 없다는 의미입니다.")
    print("   위의 상세 정보를 확인하세요.")


가설 검증: edge_pids_n의 누적 특성 확인

가설: edge_pids_n (n ≥ 2)는 fu=(n-1)에서 fu=n으로 넘어가는 시점에
      새로 발생한 엣지들만 포함하므로, fu=n 전체를 보려면
      edge_pids_1부터 edge_pids_n까지 합쳐야 함

검증 방법: 누적 edge_pids (1부터 n까지 합친 것)의 조합과 CSV 파일의 조합 비교


검증 중: n = 1
CSV 파일의 조합 개수: 55,785개
누적 edge_pids (1부터 1까지 합친 결과)의 조합 개수: 367,932개

비교 결과:
  공통 조합: 55,785개
  CSV에만 있는 조합: 0개
  누적 edge_pids에만 있는 조합: 312,147개

검증 중: n = 2
CSV 파일의 조합 개수: 74,165개
누적 edge_pids (1부터 2까지 합친 결과)의 조합 개수: 449,426개

비교 결과:
  공통 조합: 74,165개
  CSV에만 있는 조합: 0개
  누적 edge_pids에만 있는 조합: 375,261개

검증 중: n = 3
CSV 파일의 조합 개수: 85,354개
누적 edge_pids (1부터 3까지 합친 결과)의 조합 개수: 501,236개

비교 결과:
  공통 조합: 85,354개
  CSV에만 있는 조합: 0개
  누적 edge_pids에만 있는 조합: 415,882개

검증 중: n = 4
CSV 파일의 조합 개수: 94,763개
누적 edge_pids (1부터 4까지 합친 결과)의 조합 개수: 541,840개

비교 결과:
  공통 조합: 94,763개
  CSV에만 있는 조합: 0개
  누적 edge_pids에만 있는 조합: 447,077개

검증 중: n = 5
CSV 파일의 조합 개수: 102,535개
누적 edge_pids (1부터 5까지 합친 결과)의 조합 개수: 573,918개

비교 결과:
  공통 조합: 102,535개
  CSV에만 있는 조합: 0개
  누적 edge_pids에만 있는 조합: 4

In [21]:
# edge_pids_mapping_10.parquet 파일의 일부분 확인

file_path = '/home/hashjamm/results/disease_network/hr_mapping_results_fu10/edge_pids_mapping_10.parquet'

# 파일 존재 확인
if os.path.exists(file_path):
    # Parquet 파일 읽기 (엔진 명시)
    try:
        df = pd.read_parquet(file_path, engine='pyarrow')
    except Exception as e:
        print(f"pyarrow 엔진 실패: {e}")
        print("fastparquet 엔진으로 재시도...")
        df = pd.read_parquet(file_path, engine='fastparquet')
    
    print("=" * 80)
    print("edge_pids_mapping_10.parquet 파일 정보")
    print("=" * 80)
    print(f"\n전체 행 수: {len(df):,}개")
    print(f"칼럼 수: {len(df.columns)}개")
    print(f"\n칼럼 목록:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i}. {col}")
    
    print(f"\n{'='*80}")
    print("상위 10개 행:")
    print(f"{'='*80}")
    print(df.head(10).to_string())
    
    print(f"\n{'='*80}")
    print("데이터 타입:")
    print(f"{'='*80}")
    print(df.dtypes)
    
    print(f"\n{'='*80}")
    print("기본 통계 정보:")
    print(f"{'='*80}")
    print(df.describe())
    
else:
    print(f"⚠️ 파일이 존재하지 않습니다: {file_path}")

pyarrow 엔진 실패: A type extension with name pandas.period already defined
fastparquet 엔진으로 재시도...
edge_pids_mapping_10.parquet 파일 정보

전체 행 수: 12,177개
칼럼 수: 2개

칼럼 목록:
  1. key
  2. values

상위 10개 행:
          key                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [42]:
with open('/home/hashjamm/results/disease_network/edge_pids/edge_pids_1.pkl', 'rb') as f:
    edge_pids_1 = pickle.load(f)

outcome_dt_year_1 = pd.read_sas('/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_1.sas7bdat')
outcome_dt_year_1 = outcome_dt_year_1.astype({
    'PERSON_ID': 'int64',
    'KEY_SEQ': 'string',
    'abb_sick': 'string'
})

matched_a01 = pd.read_sas('/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date/matched_a01.sas7bdat')
matched_a01 = matched_a01.astype({
    'PERSON_ID': 'int64',
    'matched_id': 'int64',
    'case': 'int64',
    'index_date': 'string',
    'DTH_YM': 'string',
    'DTH_CODE1': 'string',
    'DTH_CODE2': 'string',
    'index_key_seq': 'string'
})

In [44]:
matched_a01

,PERSON_ID,matched_id,case,index_date,DTH_YM,DTH_CODE1,DTH_CODE2,index_key_seq
0,10241095,60744825,0,20030210,<NA>,<NA>,<NA>,<NA>
1,10351989,31557339,0,20030429,<NA>,<NA>,<NA>,<NA>
2,10684534,47160831,0,20031113,<NA>,<NA>,<NA>,<NA>
3,11165309,74646440,0,20030814,<NA>,<NA>,<NA>,<NA>
4,13706300,83310727,0,20030331,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...
175,82895667,82895667,1,20030825,<NA>,<NA>,<NA>,200309126976
176,83310727,83310727,1,20030331,<NA>,<NA>,<NA>,200304487730
177,84546077,84546077,1,20030728,<NA>,<NA>,<NA>,200307328399
178,86072772,86072772,1,20030722,<NA>,<NA>,<NA>,200300702841


In [8]:
def process_cause_abb_chunk(chunk_data):
    """
    각 cause_abb 청크를 처리하는 함수
    
    Args:
        chunk_data: 튜플 (cause_abbs_chunk, edge_pids_path, outcome_dt_year_path, 
                          matched_date_dir, chunk_idx, output_dir, fu)
    """
    (cause_abbs_chunk, edge_pids_path, outcome_dt_year_path, matched_date_dir, 
     chunk_idx, output_dir, fu) = chunk_data
    
    # edge_pids 데이터 로드
    with open(edge_pids_path, 'rb') as f:
        edge_pids_dict = pickle.load(f)
    
    # outcome_dt_year 데이터 로드 및 타입 변환
    outcome_dt_year_df = pd.read_sas(outcome_dt_year_path)
    outcome_dt_year_df = outcome_dt_year_df.astype({
        'PERSON_ID': 'int64',
        'KEY_SEQ': 'string',
        'abb_sick': 'string'
    })
    
    # 인덱스 설정
    outcome_index = outcome_dt_year_df.set_index(['PERSON_ID', 'abb_sick'])
    
    edge_index_key_seq_dict = {}
    edge_key_seq_dict = {}
    
    for cause_abb in cause_abbs_chunk:
        matched_file_path = os.path.join(matched_date_dir, f'matched_{cause_abb.lower()}.sas7bdat')
        
        if not os.path.exists(matched_file_path):
            continue
        
        # matched 파일 로드 및 타입 변환
        matched_df = pd.read_sas(matched_file_path)
        matched_df = matched_df.astype({
            'PERSON_ID': 'int64',
            'matched_id': 'int64',
            'case': 'int64',
            'index_date': 'string',
            'DTH_YM': 'string',
            'DTH_CODE1': 'string',
            'DTH_CODE2': 'string',
            'index_key_seq': 'string'
        })
        
        matched_case1 = matched_df[matched_df['case'] == 1]
        person_to_index_key_seq = dict(zip(matched_case1['PERSON_ID'], matched_case1['index_key_seq']))
        
        # matched_df 사용 완료 후 메모리에서 삭제
        del matched_df, matched_case1
        
        cause_combos = [(c_abb, o_abb) for (c_abb, o_abb) in edge_pids_dict.keys() if c_abb == cause_abb]
        
        for (c_abb, o_abb) in cause_combos:
            pids_array = edge_pids_dict[(c_abb, o_abb)]
            key = f'{c_abb}_{o_abb}_{fu}'
            
            pids_list = pids_array.tolist() if isinstance(pids_array, np.ndarray) else list(pids_array)
            
            index_key_seqs = [person_to_index_key_seq[pid] for pid in pids_list if pid in person_to_index_key_seq]
            if index_key_seqs:
                edge_index_key_seq_dict[key] = index_key_seqs
            
            outcome_keys = pd.MultiIndex.from_tuples([(pid, o_abb) for pid in pids_list])
            matching_keys = outcome_keys.intersection(outcome_index.index)
            if len(matching_keys) > 0:
                key_seqs = outcome_index.loc[matching_keys, 'KEY_SEQ'].tolist()
                if key_seqs:
                    edge_key_seq_dict[key] = key_seqs
        
        # person_to_index_key_seq 사용 완료 후 메모리에서 삭제
        del person_to_index_key_seq
    
    # outcome_dt_year_df와 outcome_index 사용 완료 후 메모리에서 삭제
    del outcome_dt_year_df, outcome_index
    
    # edge_pids_dict 사용 완료 후 메모리에서 삭제
    del edge_pids_dict
    
    os.makedirs(output_dir, exist_ok=True)
    
    if edge_index_key_seq_dict:
        df_index = pd.DataFrame({
            'key': list(edge_index_key_seq_dict.keys()),
            'values': list(edge_index_key_seq_dict.values())
        })
        df_index.to_parquet(
            os.path.join(output_dir, f'edge_index_key_seq_mapping_{fu}_{chunk_idx}.parquet'),
            engine='fastparquet',
            index=False
        )
        del df_index
    
    if edge_key_seq_dict:
        df_key = pd.DataFrame({
            'key': list(edge_key_seq_dict.keys()),
            'values': list(edge_key_seq_dict.values())
        })
        df_key.to_parquet(
            os.path.join(output_dir, f'edge_key_seq_mapping_{fu}_{chunk_idx}.parquet'),
            engine='fastparquet',
            index=False
        )
        del df_key
    
    # 메모리 부하를 줄이기 위해 파일만 저장하고 반환하지 않음
    return None

In [5]:
def process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=60):
    """
    청크를 생성하고 병렬 처리하는 함수
    
    Args:
        fu: follow-up 기간
        edge_pids_path: edge_pids pickle 파일 경로
        outcome_dt_year_path: outcome_dt_year SAS 파일 경로
        matched_date_dir: matched 파일들이 있는 디렉토리 경로
        output_dir: 출력 디렉토리 경로
        n_cores: 사용할 CPU 코어 수 (기본값: 60)
    """
    with open(edge_pids_path, 'rb') as f:
        edge_pids_temp = pickle.load(f)
        unique_cause_abbs = sorted(set([combo[0] for combo in edge_pids_temp.keys()]))
        del edge_pids_temp
    
    chunk_size = max(1, len(unique_cause_abbs) // n_cores)
    cause_abb_chunks = [unique_cause_abbs[i:i + chunk_size] for i in range(0, len(unique_cause_abbs), chunk_size)]
    
    os.makedirs(output_dir, exist_ok=True)
    
    chunk_data_list = [
        (chunk, edge_pids_path, outcome_dt_year_path, matched_date_dir, idx + 1, output_dir, fu) 
        for idx, chunk in enumerate(cause_abb_chunks)
    ]
    
    with Pool(processes=n_cores) as pool:
        list(tqdm(
            pool.imap(process_cause_abb_chunk, chunk_data_list),
            total=len(chunk_data_list),
            desc="Processing chunks"
        ))
    
    print(f"모든 청크 처리 완료. 결과 파일은 {output_dir}에 저장되었습니다.")

In [7]:
def merge_chunk_files(fu, output_dir, delete_chunks=False):
    """
    청크별로 나눠진 parquet 파일들을 합치는 함수
    
    Args:
        fu: follow-up 기간
        output_dir: 출력 디렉토리 경로
        delete_chunks: 합친 후 기존 청크 파일들을 삭제할지 여부 (기본값: False)
    """
    edge_index_files = sorted(glob.glob(os.path.join(output_dir, f'edge_index_key_seq_mapping_{fu}_*.parquet')))
    edge_key_files = sorted(glob.glob(os.path.join(output_dir, f'edge_key_seq_mapping_{fu}_*.parquet')))
    
    print(f"edge_index_key_seq_mapping 파일 개수: {len(edge_index_files)}개")
    print(f"edge_key_seq_mapping 파일 개수: {len(edge_key_files)}개")
    
    if edge_index_files:
        print(f"\nedge_index_key_seq_mapping 파일 합치는 중...")
        edge_index_dfs = []
        for file_path in tqdm(edge_index_files, desc="Loading edge_index files"):
            try:
                df = pd.read_parquet(file_path, engine='pyarrow')
            except:
                df = pd.read_parquet(file_path, engine='fastparquet')
            edge_index_dfs.append(df)
        
        edge_index_merged = pd.concat(edge_index_dfs, ignore_index=True)
        edge_index_output_path = os.path.join(output_dir, f'edge_index_key_seq_mapping_{fu}_dt.parquet')
        edge_index_merged.to_parquet(edge_index_output_path, engine='fastparquet', index=False)
        print(f"합친 파일 저장 완료: {edge_index_output_path}")
        print(f"  총 행 수: {len(edge_index_merged):,}개")
        del edge_index_merged, edge_index_dfs
    
    if edge_key_files:
        print(f"\nedge_key_seq_mapping 파일 합치는 중...")
        edge_key_dfs = []
        for file_path in tqdm(edge_key_files, desc="Loading edge_key files"):
            try:
                df = pd.read_parquet(file_path, engine='pyarrow')
            except:
                df = pd.read_parquet(file_path, engine='fastparquet')
            edge_key_dfs.append(df)
        
        edge_key_merged = pd.concat(edge_key_dfs, ignore_index=True)
        edge_key_output_path = os.path.join(output_dir, f'edge_key_seq_mapping_{fu}_dt.parquet')
        edge_key_merged.to_parquet(edge_key_output_path, engine='fastparquet', index=False)
        print(f"합친 파일 저장 완료: {edge_key_output_path}")
        print(f"  총 행 수: {len(edge_key_merged):,}개")
        del edge_key_merged, edge_key_dfs
    
    print(f"\n합치기 작업 완료!")
    
    if delete_chunks:
        print(f"\n기존 청크 파일들 삭제 중...")
        deleted_count = 0
        for file_path in edge_index_files + edge_key_files:
            try:
                os.remove(file_path)
                deleted_count += 1
            except Exception as e:
                print(f"삭제 실패: {file_path} - {e}")
        print(f"삭제 완료: {deleted_count}개 파일")

In [ ]:
fu = 1
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_1.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_1.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu1'

In [ ]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=60)

Processing chunks: 100%|██████████| 63/63 [1:22:59<00:00, 79.05s/it]    


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu1에 저장되었습니다.


In [ ]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 63개
edge_key_seq_mapping 파일 개수: 63개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 63/63 [00:01<00:00, 49.69it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu1/edge_index_key_seq_mapping_1.parquet
  총 행 수: 367,932개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 63/63 [00:01<00:00, 54.01it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu1/edge_key_seq_mapping_1.parquet
  총 행 수: 367,932개

합치기 작업 완료!


In [9]:
fu = 2
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_2.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_2.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu2'

In [12]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=90)

Processing chunks: 100%|██████████| 92/92 [1:00:36<00:00, 39.53s/it]  


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu2에 저장되었습니다.


In [13]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 92개
edge_key_seq_mapping 파일 개수: 92개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 92/92 [00:01<00:00, 84.70it/s] 


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu2/edge_index_key_seq_mapping_2_dt.parquet
  총 행 수: 341,239개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 92/92 [00:00<00:00, 103.18it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu2/edge_key_seq_mapping_2_dt.parquet
  총 행 수: 341,239개

합치기 작업 완료!

기존 청크 파일들 삭제 중...
삭제 완료: 184개 파일


In [9]:
fu = 3
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_3.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_3.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu3'

In [10]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=60)

Processing chunks: 100%|██████████| 63/63 [53:10<00:00, 50.65s/it]    


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu3에 저장되었습니다.


In [11]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 63개
edge_key_seq_mapping 파일 개수: 63개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 63/63 [00:00<00:00, 68.95it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu3/edge_index_key_seq_mapping_3.parquet
  총 행 수: 332,967개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 63/63 [00:00<00:00, 75.88it/s] 


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu3/edge_key_seq_mapping_3.parquet
  총 행 수: 332,967개

합치기 작업 완료!

기존 청크 파일들 삭제 중...
삭제 완료: 126개 파일


In [12]:
fu = 4
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_4.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_4.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu4'

In [13]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=90)

Processing chunks: 100%|██████████| 92/92 [53:27<00:00, 34.86s/it]    


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu4에 저장되었습니다.


In [14]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 92개
edge_key_seq_mapping 파일 개수: 92개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 92/92 [00:00<00:00, 96.26it/s] 


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu4/edge_index_key_seq_mapping_4.parquet
  총 행 수: 335,706개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 92/92 [00:00<00:00, 103.65it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu4/edge_key_seq_mapping_4.parquet
  총 행 수: 335,706개

합치기 작업 완료!

기존 청크 파일들 삭제 중...
삭제 완료: 184개 파일


In [15]:
fu = 5
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_5.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_5.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu5'

In [16]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=90)

Processing chunks: 100%|██████████| 92/92 [50:29<00:00, 32.93s/it]    


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu5에 저장되었습니다.


In [17]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 92개
edge_key_seq_mapping 파일 개수: 92개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 92/92 [00:00<00:00, 114.33it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu5/edge_index_key_seq_mapping_5.parquet
  총 행 수: 331,601개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 92/92 [00:00<00:00, 111.27it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu5/edge_key_seq_mapping_5.parquet
  총 행 수: 331,601개

합치기 작업 완료!

기존 청크 파일들 삭제 중...
삭제 완료: 184개 파일


In [18]:
fu = 6
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_6.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_6.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu6'

In [19]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=90)

Processing chunks: 100%|██████████| 92/92 [50:19<00:00, 32.82s/it]    


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu6에 저장되었습니다.


In [20]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 92개
edge_key_seq_mapping 파일 개수: 92개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 92/92 [00:00<00:00, 114.97it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu6/edge_index_key_seq_mapping_6.parquet
  총 행 수: 328,097개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 92/92 [00:00<00:00, 121.99it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu6/edge_key_seq_mapping_6.parquet
  총 행 수: 328,097개

합치기 작업 완료!

기존 청크 파일들 삭제 중...
삭제 완료: 184개 파일


In [21]:
fu = 7
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_7.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_7.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu7'

In [22]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=90)

Processing chunks:   0%|          | 0/92 [00:00<?, ?it/s]

Processing chunks: 100%|██████████| 92/92 [45:01<00:00, 29.36s/it]    


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu7에 저장되었습니다.


In [23]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 92개
edge_key_seq_mapping 파일 개수: 92개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 92/92 [00:00<00:00, 120.09it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu7/edge_index_key_seq_mapping_7.parquet
  총 행 수: 323,245개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 92/92 [00:00<00:00, 128.97it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu7/edge_key_seq_mapping_7.parquet
  총 행 수: 323,245개

합치기 작업 완료!

기존 청크 파일들 삭제 중...
삭제 완료: 184개 파일


In [24]:
fu = 8
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_8.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_8.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu8'

In [25]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=90)

Processing chunks:   0%|          | 0/92 [00:00<?, ?it/s]

Processing chunks: 100%|██████████| 92/92 [49:40<00:00, 32.40s/it]    


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu8에 저장되었습니다.


In [26]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 92개
edge_key_seq_mapping 파일 개수: 92개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 92/92 [00:00<00:00, 114.70it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu8/edge_index_key_seq_mapping_8.parquet
  총 행 수: 330,057개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 92/92 [00:00<00:00, 121.38it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu8/edge_key_seq_mapping_8.parquet
  총 행 수: 330,057개

합치기 작업 완료!

기존 청크 파일들 삭제 중...
삭제 완료: 184개 파일


In [27]:
fu = 9
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_9.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_9.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu9'

In [28]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=90)

Processing chunks:   0%|          | 0/92 [00:00<?, ?it/s]

Processing chunks: 100%|██████████| 92/92 [47:16<00:00, 30.84s/it]    


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu9에 저장되었습니다.


In [29]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 92개
edge_key_seq_mapping 파일 개수: 92개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 92/92 [00:00<00:00, 109.27it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu9/edge_index_key_seq_mapping_9.parquet
  총 행 수: 326,263개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 92/92 [00:00<00:00, 107.90it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu9/edge_key_seq_mapping_9.parquet
  총 행 수: 326,263개

합치기 작업 완료!

기존 청크 파일들 삭제 중...
삭제 완료: 184개 파일


In [30]:
fu = 10
edge_pids_path = '/home/hashjamm/results/disease_network/edge_pids/edge_pids_10.pkl'
outcome_dt_year_path = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/outcome_dt_year_10.sas7bdat'
matched_date_dir = '/home/hashjamm/project_data/disease_network/sas_files/hr_project/matched_date'
output_dir = '/home/hashjamm/results/disease_network/rr_mapping_results_fu10'

In [31]:
process_chunks(fu, edge_pids_path, outcome_dt_year_path, matched_date_dir, output_dir, n_cores=90)

Processing chunks:   0%|          | 0/92 [00:00<?, ?it/s]

Processing chunks: 100%|██████████| 92/92 [41:55<00:00, 27.34s/it]    


모든 청크 처리 완료. 결과 파일은 /home/hashjamm/results/disease_network/rr_mapping_results_fu10에 저장되었습니다.


In [32]:
merge_chunk_files(fu, output_dir, delete_chunks=True)

edge_index_key_seq_mapping 파일 개수: 92개
edge_key_seq_mapping 파일 개수: 92개

edge_index_key_seq_mapping 파일 합치는 중...


Loading edge_index files: 100%|██████████| 92/92 [00:00<00:00, 112.60it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu10/edge_index_key_seq_mapping_10.parquet
  총 행 수: 320,428개

edge_key_seq_mapping 파일 합치는 중...


Loading edge_key files: 100%|██████████| 92/92 [00:00<00:00, 119.90it/s]


합친 파일 저장 완료: /home/hashjamm/results/disease_network/rr_mapping_results_fu10/edge_key_seq_mapping_10.parquet
  총 행 수: 320,428개

합치기 작업 완료!

기존 청크 파일들 삭제 중...
삭제 완료: 184개 파일


In [3]:
def convert_edge_pids_to_parquet(fu, edge_pids_path, output_dir):
    """
    edge_pids pkl 파일을 mapping parquet 형식으로 변환하는 함수
    
    Args:
        fu: follow-up 기간
        edge_pids_path: edge_pids pickle 파일 경로
        output_dir: 출력 디렉토리 경로
    """
    with open(edge_pids_path, 'rb') as f:
        edge_pids = pickle.load(f)
    
    data_list = []
    for (cause_abb, outcome_abb), pids_array in tqdm(edge_pids.items(), desc=f"Converting fu={fu}"):
        key = f'{cause_abb}_{outcome_abb}_{fu}'
        pids_list = pids_array.tolist() if isinstance(pids_array, np.ndarray) else list(pids_array)
        data_list.append({'key': key, 'values': pids_list})
    
    del edge_pids
    
    df = pd.DataFrame(data_list)
    del data_list
    
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f'edge_pids_mapping_{fu}_dt.parquet')
    df.to_parquet(output_path, engine='fastparquet', index=False)
    
    row_count = len(df)
    del df
    
    print(f"fu={fu}: {row_count:,}개 조합 저장 완료 -> {output_path}")

edge_pids_base_dir = '/home/hashjamm/results/disease_network/edge_pids'
output_base_dir = '/home/hashjamm/results/disease_network'

for fu in range(1, 11):
    edge_pids_path = os.path.join(edge_pids_base_dir, f'edge_pids_{fu}.pkl')
    output_dir = os.path.join(output_base_dir, f'rr_mapping_results_fu{fu}')
    
    if os.path.exists(edge_pids_path):
        convert_edge_pids_to_parquet(fu, edge_pids_path, output_dir)
    else:
        print(f"⚠️ 파일이 존재하지 않습니다: {edge_pids_path}")

print("\n모든 변환 작업 완료!")

Converting fu=1: 100%|██████████| 367932/367932 [00:00<00:00, 505095.79it/s]


fu=1: 367,932개 조합 저장 완료 -> /home/hashjamm/results/disease_network/rr_mapping_results_fu1/edge_pids_mapping_1_dt.parquet


Converting fu=2: 100%|██████████| 341239/341239 [00:01<00:00, 325470.06it/s]


fu=2: 341,239개 조합 저장 완료 -> /home/hashjamm/results/disease_network/rr_mapping_results_fu2/edge_pids_mapping_2_dt.parquet


Converting fu=3: 100%|██████████| 332967/332967 [00:01<00:00, 298493.24it/s]


...

fu=8: 330,057개 조합 저장 완료 -> /home/hashjamm/results/disease_network/rr_mapping_results_fu8/edge_pids_mapping_8_dt.parquet


Converting fu=9: 100%|██████████| 326263/326263 [00:00<00:00, 403740.17it/s]


fu=9: 326,263개 조합 저장 완료 -> /home/hashjamm/results/disease_network/rr_mapping_results_fu9/edge_pids_mapping_9_dt.parquet


Converting fu=10: 100%|██████████| 320428/320428 [00:00<00:00, 354954.89it/s]


fu=10: 320,428개 조합 저장 완료 -> /home/hashjamm/results/disease_network/rr_mapping_results_fu10/edge_pids_mapping_10_dt.parquet

모든 변환 작업 완료!


In [4]:
def create_all_cumulative_mappings(base_dir='/home/hashjamm/results/disease_network'):
    """
    모든 dt 파일을 메모리에 올린 후 각 fu에 대해 누적 매핑 데이터를 생성하는 함수
    
    Args:
        base_dir: 기본 디렉토리 경로
    """
    mapping_types = [
        'edge_index_key_seq_mapping',
        'edge_key_seq_mapping',
        'edge_pids_mapping'
    ]
    
    for mapping_type in mapping_types:
        print(f"\n{'='*80}")
        print(f"{mapping_type} 처리 시작")
        print(f"{'='*80}")
        
        dt_data = {}
        
        print(f"모든 dt 파일 로드 중...")
        for fu in tqdm(range(1, 11), desc="  dt 파일 로드"):
            source_dir = os.path.join(base_dir, f'rr_mapping_results_fu{fu}')
            dt_file_path = os.path.join(source_dir, f'{mapping_type}_{fu}_dt.parquet')
            
            if not os.path.exists(dt_file_path):
                dt_data[fu] = {}
                continue
            
            try:
                df = pd.read_parquet(dt_file_path, engine='pyarrow')
            except:
                df = pd.read_parquet(dt_file_path, engine='fastparquet')
            
            fu_data = {}
            for _, row in df.iterrows():
                fu_data[row['key']] = row['values']
            
            dt_data[fu] = fu_data
            del df
        
        print(f"각 fu에 대해 누적 매핑 데이터 생성 중...")
        for target_fu in tqdm(range(1, 11), desc="  누적 데이터 생성"):
            output_dir = os.path.join(base_dir, f'rr_mapping_results_fu{target_fu}')
            os.makedirs(output_dir, exist_ok=True)
            
            merged_data = {}
            
            for current_fu in range(1, target_fu + 1):
                if current_fu not in dt_data or not dt_data[current_fu]:
                    continue
                
                for key, values in dt_data[current_fu].items():
                    if key not in merged_data:
                        merged_data[key] = []
                    
                    if isinstance(values, list):
                        merged_data[key].extend(values)
                    else:
                        merged_data[key].append(values)
            
            if not merged_data:
                continue
            
            result_list = []
            for key, values_list in merged_data.items():
                unique_values = list(set(values_list))
                result_list.append({'key': key, 'values': unique_values})
            
            del merged_data
            
            result_df = pd.DataFrame(result_list)
            del result_list
            
            output_path = os.path.join(output_dir, f'{mapping_type}_{target_fu}.parquet')
            result_df.to_parquet(output_path, engine='fastparquet', index=False)
            
            del result_df
        
        del dt_data
        print(f"{mapping_type} 처리 완료!")

create_all_cumulative_mappings()

print("\n모든 누적 매핑 데이터 생성 완료!")


edge_index_key_seq_mapping 처리 시작
모든 dt 파일 로드 중...


  dt 파일 로드: 100%|██████████| 10/10 [01:32<00:00,  9.28s/it]


각 fu에 대해 누적 매핑 데이터 생성 중...


  누적 데이터 생성: 100%|██████████| 10/10 [02:19<00:00, 13.91s/it]


edge_index_key_seq_mapping 처리 완료!

edge_key_seq_mapping 처리 시작
모든 dt 파일 로드 중...


...

edge_key_seq_mapping 처리 완료!

edge_pids_mapping 처리 시작
모든 dt 파일 로드 중...


  dt 파일 로드: 100%|██████████| 10/10 [01:31<00:00,  9.20s/it]


각 fu에 대해 누적 매핑 데이터 생성 중...


  누적 데이터 생성: 100%|██████████| 10/10 [02:03<00:00, 12.36s/it]

edge_pids_mapping 처리 완료!

모든 누적 매핑 데이터 생성 완료!


In [5]:
def delete_dt_files(base_dir='/home/hashjamm/results/disease_network'):
    """
    각 폴더별 dt 파일들을 삭제하는 함수
    
    Args:
        base_dir: 기본 디렉토리 경로
    """
    mapping_types = [
        'edge_index_key_seq_mapping',
        'edge_key_seq_mapping',
        'edge_pids_mapping'
    ]
    
    total_deleted = 0
    
    for fu in range(1, 11):
        folder_path = os.path.join(base_dir, f'rr_mapping_results_fu{fu}')
        
        if not os.path.exists(folder_path):
            continue
        
        print(f"\nfu={fu} 폴더 처리 중: {folder_path}")
        deleted_count = 0
        
        for mapping_type in mapping_types:
            dt_file_pattern = os.path.join(folder_path, f'{mapping_type}_{fu}_dt.parquet')
            
            if os.path.exists(dt_file_pattern):
                try:
                    os.remove(dt_file_pattern)
                    deleted_count += 1
                    print(f"  삭제: {os.path.basename(dt_file_pattern)}")
                except Exception as e:
                    print(f"  삭제 실패: {os.path.basename(dt_file_pattern)} - {e}")
        
        if deleted_count == 0:
            print(f"  dt 파일이 없습니다.")
        else:
            print(f"  총 {deleted_count}개 파일 삭제 완료")
            total_deleted += deleted_count
    
    print(f"\n{'='*80}")
    print(f"전체 삭제 완료: 총 {total_deleted}개 dt 파일 삭제됨")
    print(f"{'='*80}")

delete_dt_files()



fu=1 폴더 처리 중: /home/hashjamm/results/disease_network/rr_mapping_results_fu1
  삭제: edge_index_key_seq_mapping_1_dt.parquet
  삭제: edge_key_seq_mapping_1_dt.parquet
  삭제: edge_pids_mapping_1_dt.parquet
  총 3개 파일 삭제 완료

fu=2 폴더 처리 중: /home/hashjamm/results/disease_network/rr_mapping_results_fu2
  삭제: edge_index_key_seq_mapping_2_dt.parquet
  삭제: edge_key_seq_mapping_2_dt.parquet
  삭제: edge_pids_mapping_2_dt.parquet
  총 3개 파일 삭제 완료

fu=3 폴더 처리 중: /home/hashjamm/results/disease_network/rr_mapping_results_fu3
  삭제: edge_index_key_seq_mapping_3_dt.parquet
  삭제: edge_key_seq_mapping_3_dt.parquet
  삭제: edge_pids_mapping_3_dt.parquet
  총 3개 파일 삭제 완료

fu=4 폴더 처리 중: /home/hashjamm/results/disease_network/rr_mapping_results_fu4
  삭제: edge_index_key_seq_mapping_4_dt.parquet
  삭제: edge_key_seq_mapping_4_dt.parquet
  삭제: edge_pids_mapping_4_dt.parquet
  총 3개 파일 삭제 완료

fu=5 폴더 처리 중: /home/hashjamm/results/disease_network/rr_mapping_results_fu5
  삭제: edge_index_key_seq_mapping_5_dt.parquet
  삭제: edge_ke